# 1.缓冲窗口记忆示例.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
@Time    : 2024/5/23 1:55
@Author  : thezehui@gmail.com
@File    : 1.缓冲窗口记忆示例.py
"""
from operator import itemgetter

import dotenv
from langchain.memory import ConversationTokenBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

# 1.创建提示模板&记忆
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是OpenAI开发的聊天机器人，请根据对应的上下文回复用户问题"),
    MessagesPlaceholder("history"),  # 需要的history其实是一个列表
    ("human", "{query}"),
])
memory = ConversationTokenBufferMemory(
    return_messages=True,
    input_key="query",
    llm=ChatOpenAI()
)

# 2.创建大语言模型
llm = ChatOpenAI(model="gpt-4o")

# 3.构建链应用
chain = RunnablePassthrough.assign(
    history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
) | prompt | llm | StrOutputParser()

# 4.死循环构建对话命令行
while True:
    query = input("Human: ")

    if query == "q":
        exit(0)

    chain_input = {"query": query, "language": "中文"}

    response = chain.stream(chain_input)
    print("AI: ", flush=True, end="")
    output = ""
    for chunk in response:
        output += chunk
        print(chunk, flush=True, end="")
    memory.save_context(chain_input, {"output": output})
    print("")
    print("history: ", memory.load_memory_variables({}))

AI: 你好，陈同学！很高兴认识你～有什么我可以帮忙的吗？ 😊
history:  {'history': [HumanMessage(content='你好我是陈同学'), AIMessage(content='你好，陈同学！很高兴认识你～有什么我可以帮忙的吗？ 😊')]}
AI: 你刚刚告诉我你是陈同学，所以我猜你姓陈。不过，你的具体名字你还没告诉我呢！😊
history:  {'history': [HumanMessage(content='你好我是陈同学'), AIMessage(content='你好，陈同学！很高兴认识你～有什么我可以帮忙的吗？ 😊'), HumanMessage(content='我叫什么名字'), AIMessage(content='你刚刚告诉我你是陈同学，所以我猜你姓陈。不过，你的具体名字你还没告诉我呢！😊')]}
AI: 你好！有什么需要帮助的吗？ 😊
history:  {'history': [HumanMessage(content='你好我是陈同学'), AIMessage(content='你好，陈同学！很高兴认识你～有什么我可以帮忙的吗？ 😊'), HumanMessage(content='我叫什么名字'), AIMessage(content='你刚刚告诉我你是陈同学，所以我猜你姓陈。不过，你的具体名字你还没告诉我呢！😊'), HumanMessage(content=''), AIMessage(content='你好！有什么需要帮助的吗？ 😊')]}
AI: 你好！如果有任何问题或想聊的事情，随时告诉我哦～ 😊
history:  {'history': [HumanMessage(content='你好我是陈同学'), AIMessage(content='你好，陈同学！很高兴认识你～有什么我可以帮忙的吗？ 😊'), HumanMessage(content='我叫什么名字'), AIMessage(content='你刚刚告诉我你是陈同学，所以我猜你姓陈。不过，你的具体名字你还没告诉我呢！😊'), HumanMessage(content=''), AIMessage(content='你好！有什么需要帮助的吗？ 😊'), HumanMessage(content=''), AIMessage(cont